In [1]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator

## Preprocessing test and train data

In [2]:

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 5,
                                   shear_range = 0.2,
                                   validation_split=0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory(r"C:\Users\DEVANSH\OneDrive\Desktop\smoke train",
                                                 target_size = (224, 224),
                                                 subset="training",
                                                 batch_size = 20,
                                                 class_mode = 'binary')
validation_set = train_datagen.flow_from_directory(r"C:\Users\DEVANSH\OneDrive\Desktop\smoke train",
                                                 target_size = (224, 224),
                                                 subset="validation",
                                                 batch_size = 20,
                                                 class_mode = 'binary')

Found 912 images belonging to 2 classes.
Found 227 images belonging to 2 classes.


In [3]:
class_names = ["no smoke","smoke"]

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(r"C:\Users\DEVANSH\OneDrive\Desktop\smoke test",
                                            target_size = (224, 224),
                                            batch_size = 20,
                                            class_mode = 'binary')

Found 267 images belonging to 2 classes.


## Creating a model

In [5]:
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [keras.layers.Conv2D(filters, 3, strides=strides,padding="same", use_bias=False),
                            keras.layers.BatchNormalization(),
                            self.activation,keras.layers.Conv2D(filters, 3, strides=1,padding="same", use_bias=False), 
                            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [keras.layers.Conv2D(filters, 1, strides=strides,padding="same", use_bias=False),
                                keras.layers.BatchNormalization()]
def call(self, inputs):
    Z = inputs
    for layer in self.main_layers:
        Z = layer(Z)
    skip_Z = inputs
    for layer in self.skip_layers:
        skip_Z = layer(skip_Z)
    return self.activation(Z + skip_Z)

In [6]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, 7, strides=2, input_shape=[224, 224, 3],padding="same", use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1, activation="softmax"))

## Compiling model

In [7]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

In [8]:
model.compile(optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

## training model

In [9]:
model.fit(training_set, epochs = 40, validation_data = validation_set)

Epoch 1/40
46/46 [==============================] - 45s 940ms/step - loss: 0.4603 - accuracy: 0.8059 - val_loss: 0.6134 - val_accuracy: 0.8062
Epoch 2/40
46/46 [==============================] - 38s 830ms/step - loss: 0.4053 - accuracy: 0.8059 - val_loss: 0.5959 - val_accuracy: 0.8062
Epoch 3/40
46/46 [==============================] - 48s 1s/step - loss: 0.3850 - accuracy: 0.8059 - val_loss: 0.5524 - val_accuracy: 0.8062
Epoch 4/40
46/46 [==============================] - 38s 832ms/step - loss: 0.3794 - accuracy: 0.8059 - val_loss: 0.5149 - val_accuracy: 0.8062
Epoch 5/40
46/46 [==============================] - 37s 813ms/step - loss: 0.3734 - accuracy: 0.8059 - val_loss: 0.4934 - val_accuracy: 0.8062
Epoch 6/40
46/46 [==============================] - 37s 800ms/step - loss: 0.3581 - accuracy: 0.8059 - val_loss: 0.4445 - val_accuracy: 0.8062
Epoch 7/40
46/46 [==============================] - 37s 803ms/step - loss: 0.3635 - accuracy: 0.8059 - val_loss: 0.4354 - val_accuracy: 0.8062
Ep